In [1]:
#@title Get text train and test ,X and Y
nrows_train = 20000 # @param {type:"integer"}
sequence_length = 13 # @param {type:"integer"}

feed_lenght = 8

import pandas as pd
import numpy as np
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
import json
from keras.preprocessing.text import tokenizer_from_json

#load fusion
fus = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/gpt/LECO/LECOfusion1.csv",
                  parse_dates=["time"], index_col="time")

#Get the train and test train
texts_train = fus["fusion"].sample(nrows_train,)
texts_test = fus["fusion"].drop(texts_train.index)

#save texts test
texts_test.to_csv("/content/drive/MyDrive/Colab Notebooks/gpt/LECO/LECOtexts_test1.csv")

# Tokenize text
tokenizer = Tokenizer()

#tokenizer.fit_on_texts(texts_train)
tokenizer.fit_on_texts(fus["fusion"])

#Save tokenizer
tokenizer_json = tokenizer.to_json()

# Save the JSON configuration to a file
with open('/content/drive/MyDrive/Colab Notebooks/gpt/LECO/LECOtokenizer_config1.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

sequences = tokenizer.texts_to_sequences(texts_train)

# Prepare input and output data
X = []
y = []
for sequence in sequences:
    for i in range(1, len(sequence)):
        x_seq = sequence[:i]
        x_seq_padded = pad_sequences([x_seq], maxlen=sequence_length, padding='pre')
        X.append(x_seq_padded[0])
        y.append(sequence[i])

X = np.array(X)
y = np.array(y)

#filter seed/model words
df = pd.DataFrame(X)
df["y"] = y
df_fil =  df[(df.iloc[:, :sequence_length-feed_lenght+1] != 0).any(axis=1)]
X = df_fil.iloc[:, :-1].values
y = df_fil.iloc[:, -1].values

# One hot encode the outputs
y = np.eye(len(tokenizer.word_index) + 1)[y]

print("Dictionary size (words): len(y[i]):",len(y[1]) )
print("Total X variables:",len(X) )

print("\nText train example1")
display(texts_train[0])

print("\nSequences example 1")
display(sequences[0])

print("\nText train example2")
display(texts_train[1])

print("\nSequences example 2")
display(sequences[1])

print("\nX")
display(X[:25])

print("\nDictionary first 10 words")
display({k: tokenizer.word_index[k] for k in list(tokenizer.word_index)[:10]})


Dictionary size (words): len(y[i]): 3397
Total X variables: 163223

Text train example1


'20015kt 9999 prec0y CL2 CM5 18 15 q1013 19010KT 9999 FEW030 19/16 Q1015 NOSIG'


Sequences example 1


[261, 1, 20, 45, 76, 18, 9, 43, 675, 1, 54, 28, 12, 32, 4]


Text train example2


...

[520, 1, 2, 5, 80, 57, 12, 49, 1346, 1, 26, 71, 44, 9, 53, 17, 919, 1181]


X


array([[   0,    0,    0,    0,    0,  261,    1,   20,   45,   76,   18,
           9,   43],
       [   0,    0,    0,    0,  261,    1,   20,   45,   76,   18,    9,
          43,  675],
       [   0,    0,    0,  261,    1,   20,   45,   76,   18,    9,   43,
         675,    1],
       [   0,    0,  261,    1,   20,   45,   76,   18,    9,   43,  675,
           1,   54],
       [   0,  261,    1,   20,   45,   76,   18,    9,   43,  675,    1,
          54,   28],
       [ 261,    1,   20,   45,   76,   18,    9,   43,  675,    1,   54,
          28,   12],
       [   1,   20,   45,   76,   18,    9,   43,  675,    1,   54,   28,
          12,   32],
       [   0,    0,    0,    0,    0,  520,    1,    2,    5,   80,   57,
          12,   49],
       [   0,    0,    0,    0,  520,    1,    2,    5,   80,   57,   12,
          49, 1346],
       [   0,    0,    0,  520,    1,    2,    5,   80,   57,   12,   49,
        1346,    1],
       [   0,    0,  520,    1,    2,    5,   80, 


Dictionary first 10 words


{'9999': 1,
 'prec0n': 2,
 'cm0': 3,
 'nosig': 4,
 'cl0': 5,
 '12': 6,
 '13': 7,
 '14': 8,
 '15': 9,
 '11': 10}

In [2]:
#@title Train the model

from keras.layers import Bidirectional

# Build the LSTM model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=20, input_length=sequence_length),
    Bidirectional(LSTM(130)),

    Dense(len(tokenizer.word_index)+1, activation='softmax')
])


# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=50,batch_size=512)

#save the model
model.save("/content/drive/MyDrive/Colab Notebooks/gpt/LECO/LECOmodel1.keras")


Epoch 1/50
319/319 [==============================] - 25s 69ms/step - loss: 4.0193 - accuracy: 0.1977
Epoch 2/50
319/319 [==============================] - 22s 68ms/step - loss: 2.9292 - accuracy: 0.2807
Epoch 3/50
319/319 [==============================] - 22s 68ms/step - loss: 2.6271 - accuracy: 0.3198
Epoch 4/50
319/319 [==============================] - 22s 69ms/step - loss: 2.4860 - accuracy: 0.3383
Epoch 5/50
319/319 [==============================] - 22s 70ms/step - loss: 2.3960 - accuracy: 0.3521
Epoch 6/50
319/319 [==============================] - 22s 68ms/step - loss: 2.3279 - accuracy: 0.3625
Epoch 7/50
319/319 [==============================] - 22s 68ms/step - loss: 2.2710 - accuracy: 0.3728
Epoch 8/50
319/319 [==============================] - 22s 69ms/step - loss: 2.2157 - accuracy: 0.3809
Epoch 9/50
319/319 [==============================] - 22s 69ms/step - loss: 2.1702 - accuracy: 0.3893
Epoch 10/50
319/319 [==============================] - 22s 68ms/step - loss: 2.115